<section id="title-slide">
  <h1 class="title">The ABC of Computational Text Analysis</h1>
  <h2 class="subtitle">#11: Working with texts</h2>
  <p class="author">Alex Flückiger</p><p class="date">11/25 May 2023</p>
</section>

## Working with Texts
Texts are represented as strings of any length.


In [8]:
sentence_1 = "I love NLP and social science"
sentence_2 = "Computational Social Science applies NLP to question of social questions."

text = sentence_1 + " " + sentence_2
text

'I love NLP and social science Computational Social Science applies NLP to question of social questions.'

## Text Modifactions

In [9]:
# replace `.` with `!`
text.replace(".", "!")

# change text to lowercased letters
text.lower()

# split text at space (~words)
text.split(" ")


['I',
 'love',
 'NLP',
 'and',
 'social',
 'science',
 'Computational',
 'Social',
 'Science',
 'applies',
 'NLP',
 'to',
 'question',
 'of',
 'social',
 'questions.']

## Count Words

In [10]:
from collections import Counter

# initialize a counter object
counter = Counter()

# split the text and pass all elemnents (~words) to the counter
counter.update(text.split(" "))

# get the three most common words
counter.most_common(3)


[('NLP', 2), ('social', 2), ('I', 1)]

## Read from Textfiles

In [11]:
from pathlib import Path

infile = Path("../data/swiss_party_programmes/txt/gruene_programmes/gruene_programme_2019.txt")

text = infile.read_text()

# show first 100 characters of file 
print(text[0:100])


IMPRESSUM
GRÜNE Schweiz
Waisenhausplatz 21
3011 Bern
Tel. 031 326 66 00
www.gruene.ch
gruene@gruene.


## Write into Textfiles

In [12]:
# write content to file
outfile = Path("../data/scanned_pdf_sample/fdp_scan_party_programme_1947_lowercased.txt")

text = text.lower()

# write to file
with outfile.open("w") as f:
    f.write(text)


## Counting Words in a Textfile

In [23]:
from pathlib import Path
import re

infile = Path("../data/swiss_party_programmes/txt/gruene_programmes/gruene_programme_2019.txt")
text = infile.read_text()

# replace (multiple) newlines with single space
text = re.sub(r"\n+", " ", text)

# keep only alphanumeric characters and whitespace
text = re.sub(r"[^\w\s]", "", text)

# lowercase all text
text = text.lower()
vocab = Counter(text.split(" "))

# write to file, one word and its frequency per line
outfile = Path("../analysis/vocab_frq_test.txt")
with outfile.open("w") as f:
    for word, frq in vocab.most_common():
        line = f"{word}\t{frq}\n"
        f.write(line)

vocab.most_common(5)


[('und', 595), ('die', 517), ('der', 394), ('', 277), ('für', 217)]

## PDF: Digitalized or Digital?

### Two flavours of PDF documents
![Digitalized PDF made from a scanned page](../../lectures/images/pdf_scan.png)
![Native PDF converted from digital document (e.g., docx)](../../lectures/images/pdf_digital.png)

## Conversion of a single native PDFs

### Use case: [Swiss party programmes](https://visuals.manifesto-project.wzb.eu/mpdb-shiny/cmp_dashboard_dataset/)


In [17]:
from pypdf import PdfReader

# path to PDF file
pdf_path = Path("../data/swiss_party_programmes/pdf/gruene_programmes/gruene_programme_2019.pdf")

reader = PdfReader(pdf_path)

for page in reader.pages:
    text_page = page.extract_text()
    text += " " + text_page

print(text[:500])


impressum grüne schweiz waisenhausplatz 21 3011 bern tel 031 326 66 00 wwwgruenech gruenegruenech postkonto 80267473 wahlplattform 2019  2023 beschlossen an der delegiertenversammlung vom 12 januar 2019 in emmen ergänzt durch den resolutionsbeschluss der delegiertenversammlung vom 6 april 2019 in sierre wahlplattform der grünen schweiz 2019  2023 ii inhaltsverzeichnis genderneutrale sprache 4 einleitung 5 wofür wir uns einsetzen 6 1 1 die grünen sind die partei der umwelt 11 für die umsetzung de


## Optical Character Recognition (OCR)


- OCR ~ convert images into text
  - extract text from scans/images
- `tesseract` performs OCR
  - language-specific models
  - supports handwriting + Fraktur texts
- image quality is crucial

<img src="../../lectures/images/ocr.png" alt="Steps when performing OCR" style="width: 500px;"/>

## Conversion of a single digitalized PDF

### use-case: [historical party programmes](https://visuals.manifesto-project.wzb.eu/mpdb-shiny/cmp_dashboard_dataset/)

1. extract image from PDF
2. run optical character recognition (OCR) on the image

In [19]:
import pytesseract
from pdf2image import convert_from_path

# path to PDF file
pdf_path = Path("../data/scanned_pdf_sample/fdp_scan_party_programme_1947.pdf")

# convert PDF to images (one image per page)
pages = convert_from_path(pdf_path, fmt="png")

# initialize text to collect the text per page
text = ""

# iterate over pages
for pageNum,imgBlob in enumerate(pages):
    # extract text from image per page
    text_page = pytesseract.image_to_string(imgBlob, lang='deu')

    # append text for each page
    text += " " + text_page

print(text[:100])

 G3 420 47

FREIHEIT
FÜR UNSERE ZEIT

Vorschläge zur Orientierung
der freisinnigen Politik

nach dem


## Extract the text from all PDFs in a folder

In [ ]:
# path to PDF directory
indir = Path("../data/scanned_pdf_sample/")
outdir = Path("../data/scanned_pdf_sample/extracted")

# create output folder if it does not exist
outdir.mkdir(parents=True, exist_ok=True)

# iterate over all PDFs
for infile in indir.glob(pattern="*.pdf"):
    print(f"Reading PDF file: {infile}")
    
    pages = convert_from_path(infile, fmt="png")
    text = ""
    
    for pageNum,imgBlob in enumerate(pages):
        text_page = pytesseract.image_to_string(imgBlob, lang='deu')
        text += " " + text_page

    # define name of outfile (name.pdf -> name.txt)
    outfile = outdir / (infile.stem + ".txt")

    # write content to file
    with outfile.open("w") as f:
        f.write(text)
    
    print(f"Extracted text to: {outfile}")

## Bonus: Clean up Artefacts

- remove empty lines
- remove page numbers
- remove footer
- merge hyphenated words

In [24]:
# Remove multiple lines in a string using regular expressions

import re

text = """
This is an example Text.

YOUR_PATTERN REMOVE THIS

Keep this and the following.
"""

# remove a multiline string by substituting the match with an empty string
# re.DOTALL make the . matching the newline character \n
text_clean = re.sub("YOUR_PATTERN.*\n\n", "", text, re.DOTALL)

print(text_clean)


This is an example Text.

Keep this and the following.

